In [1]:
import re
import requests
from bs4 import BeautifulSoup

In [2]:
import csv

In [3]:
url = r'https://www.alcaldiabogota.gov.co/sisjur/normas/Norma1.jsp?i=4162'

In [4]:
response = requests.get(url)
print(response)

<Response [200]>


In [5]:
if response.status_code == 200:
    # Crear un objeto BeautifulSoup para analizar el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Extraer el texto de la página (sin etiquetas HTML)
    texto = soup.get_text()
    
    # Mostrar el texto extraído
    print(texto)
else:
    print(f'Error al realizar la solicitud: {response.status_code}')













Ley  675 de 2001 Congreso de la República de Colombia


















RÉGIMEN LEGAL DE BOGOTÁ D.C.
© Propiedad de la Secretaría Jurídica Distrital de la Alcaldía Mayor de Bogotá D.C.



						Secretaría  Jurídica Distrital
					









Cargando el Contenido del Documento


Por favor espere...







Ley  675 de 2001 Congreso de la República de Colombia





					Fecha de Expedición: 
				
 
					03/08/2001
				



					Fecha de Entrada en Vigencia:
				

					04/08/2001
				



					Medio de Publicación:  
				

					Diario Oficial 44.509 del 4 de agosto de 2001
				



					Temas
				




		La Secretaría Jurídica Distrital aclara que la información aquí contenida tiene exclusivamente carácter informativo, 
		su vigencia está sujeta al análisis y competencias que determine la Ley o los reglamentos. Los contenidos están en
		 permanente actualización.
		

   


 








LEY 675 DE 2001
(Agosto 03)
Por medio de la cual se expide el régimen de propiedad horizontalVer modi

## Referencia

In [59]:
def extraer_articulos(texto):
    
    patron_articulo = re.compile(r'(Artículo|ARTÍCULO|ARTICULO)\s+(\d{1,2})\s*º?\.? ([\s\S]*?)(?=\s*(?:CAPÍTULO|ARTÍCULO|Artículo|$|[\r\n]{2}))', re.DOTALL)
    patron_relacion = re.compile(r'(Artículo|ARTÍCULO|ARTICULO)\s+(\d{1,2})\s*º?\.\s*([^\.]+?)\.')

    lista_articulos = []
    
    try:
        # Buscar todos los artículos en el texto
        articulos = patron_articulo.findall(texto)
        
        for titulo, num, contenido in articulos:
            # Buscar la relación en el contenido completo del artículo
            relacion_match = patron_relacion.search(f"{titulo} {num}. {contenido}")
            
            if relacion_match:
                relacion = relacion_match.group(3).strip()
                # Excluir la relación del contenido
                contenido = contenido.replace(relacion, '').strip()
                # Eliminar el punto extra si queda al principio del contenido
                contenido = re.sub(r'^\.\s*', '', contenido)
            else:
                relacion = ''
            
            # Crear el diccionario para el artículo
            articulo_dict = {
                'numero_articulo': int(num),
                'name': f'{titulo.capitalize()} {num}',
                'relacion': relacion,
                'contenido': contenido.strip()
            }
            
            lista_articulos.append(articulo_dict)
    
    except Exception as e:
        print(f"Error al procesar el texto: {e}")
    
    return lista_articulos


In [6]:
def extraer_articulos_y_capitulos(texto):
    # Expresión regular para capturar capítulos
    patron_capitulo = re.compile(r'(CAPÍTULO|Capítulo|CAPITULO)\s+([IVXLCDM]{1,3})\s*\.?\s*([\s\S]*?)(?=\s*(?:CAPÍTULO|$))')
    
    # Expresión regular para capturar artículos dentro de cada capítulo
    patron_articulo = re.compile(r'(Artículo|ARTÍCULO|ARTICULO)\s+(\d{1,2})\s*º?\.?\s*([\s\S]*?)(?=\s*(?:ARTÍCULO|Artículo|CAPÍTULO|$|[\r\n]{2}))', re.DOTALL)
    
    # Expresión regular para capturar la "relación" dentro de cada artículo
    patron_relacion = re.compile(r'(Artículo|ARTÍCULO|ARTICULO)\s+(\d{1,2})\s*º?\.\s*([^\.]+?)\.')

    lista_articulos = []

    try:
        # Buscar todos los capítulos en el texto
        capitulos = patron_capitulo.findall(texto)
        
        for capitulo, num_capitulo, contenido_capitulo in capitulos:
            nombre_capitulo = f'{capitulo.capitalize()} {num_capitulo}'
            
            # Buscar todos los artículos en el contenido del capítulo
            articulos = patron_articulo.findall(contenido_capitulo)
            
            for titulo, num_articulo, contenido in articulos:
                # Buscar la relación en el contenido del artículo
                relacion_match = patron_relacion.search(f"{titulo} {num_articulo}. {contenido}")
                
                if relacion_match:
                    relacion = relacion_match.group(3).strip()
                    # Excluir la relación del contenido
                    contenido = contenido.replace(relacion, '').strip()
                    # Eliminar el punto extra si queda al principio del contenido
                    contenido = re.sub(r'^\.\s*', '', contenido)
                else:
                    relacion = ''
                
                # Crear el diccionario para el artículo
                articulo_dict = {
                    'numero_articulo': int(num_articulo),
                    'name': f'{titulo.capitalize()} {num_articulo}',
                    'relacion': relacion,
                    'contenido': contenido.strip(),
                    'capitulo': nombre_capitulo  # Agregar el nombre del capítulo
                }
                
                lista_articulos.append(articulo_dict)
    
    except Exception as e:
        print(f"Error al procesar el texto: {e}")
    
    return lista_articulos

In [30]:
def extraer_capitulos(texto):
    # Expresión regular para capturar los capítulos y su contenido
    patron_capitulo = r'\n(CAPÍTULO|Capítulo|CAPITULO)\s+([IVXLCDM]{1,4})\s*\.?\s*([^\n]*)'
    
    # Buscar todos los capítulos en el texto
    capitulos = re.findall(patron_capitulo, texto)
    
    lista_capitulos = []
    id_capitulo = 1

    # Recorremos los capítulos y los organizamos
    for capitulo, num_capitulo, tema_capitulo in capitulos:
        # Crear el diccionario para cada capítulo
        capitulo_dict = {
            'id': id_capitulo,  # Asignar un ID único a cada capítulo
            'name': f'{capitulo.capitalize()} {num_capitulo.strip()}',  # Nombre del capítulo (e.g., "Capítulo I")
            'Tema': tema_capitulo.strip()  # Tema del capítulo (e.g., "Definiciones")
        }
        
        lista_capitulos.append(capitulo_dict)
        id_capitulo += 1  # Incrementar el ID para el siguiente capítulo
    
    return lista_capitulos

## Refinado


In [95]:
# Estraer titulos
def extraer_titulos(texto):
    # Definir la expresión regular para capturar los títulos y su contenido hasta el primer salto de línea
    #Nodos
    patron = re.compile(r'\n(TITULO|Titulo)\s+([IVXLCDM]{1,3})\s*\.?\s*(.*?)(?=\n(?:TITULO|Titulo)|$)', re.DOTALL)
    
    patron_capitulo = re.compile(r'\n(CAPÍTULO|Capítulo|CAPITULO)\s+([IVXLCDM]{1,4})\s*\.?\s*(.*?)(?=\n(?:CAPÍTULO|Capítulo|CAPITULO|TITULO|Titulo)|$)', re.DOTALL)
 
    #patron_articulo = re.compile(r'(Artículo|ARTÍCULO|ARTICULO)\s+(\d{1,2})\s*º?\.?\s*([\s\S]*?)(?=\s*(?:ARTÍCULO|Artículo|CAPÍTULO|$|[\r\n]{2}))', re.DOTALL)
    patron_articulo = re.compile(r'(Artículo|ARTÍCULO|ARTICULO)\s+(\d{1,2})\s*º?\.? ([\s\S]*?)(?=\s*(?:CAPÍTULO|ARTÍCULO|Artículo|$|[\r\n]{2}))', re.DOTALL)
   
    # Buscar todos los títulos en el texto
    titulos = re.findall(patron, texto)

    # Crear una lista de diccionarios para almacenar los títulos y sus enlaces
    lista_titulos = []

    
    for titulo, num, contenido in titulos: 
        lista_capitulos = []        
        
        
        capitulos = re.findall(patron_capitulo, contenido) #Se buscan las coinciddencias con los capitulos dentro del titulo
        
        for capitulo, n, cont in capitulos:
            lista_articulos = []
            dict_capitulo = {}
            #print(cont[:10])
            articulos = re.findall(patron_articulo, cont) # Se buscan todos los articulos dentro del capitulo
            print(cont[:150])
            
            for _, num_articulo, contenido_articulo in articulos:
                #print(contenido_articulo[:10])
                dic_articulos ={}
                dic_articulos = {
                    'articulo': f'articulo {num_articulo}',
                    'contenido': contenido_articulo.strip()
                }
                lista_articulos.append(dic_articulos) 
            
            dict_capitulo = {'capitulo':f'Capitulo {n}',
                            'articulos':lista_articulos                
                            }
                
            lista_capitulos.append(dict_capitulo)  
            
        info = {
            'titulo': f'{titulo.capitalize()} {num.strip()}',
            #'enlace': contenido.strip()
            'capitulos':lista_capitulos,
            'articulos':lista_articulos
            
            }
        lista_titulos.append(info)
    print(titulos)
    return lista_titulos

# titulos

In [10]:
# Estraer titulos
def extraer_titulos(texto):
    # Definir la expresión regular para capturar los títulos y su contenido hasta el primer salto de línea
    patron = r'(?:\n)(TITULO|Titulo)\s+([IVXLCDM]{1,3})\s*\.?\s*(.*?)(?=\n)'  


    # Buscar todos los títulos en el texto
    titulos = re.findall(patron, texto)

    # Crear una lista de diccionarios para almacenar los títulos y sus enlaces
    lista_titulos = []

    # Iterar sobre las coincidencias encontradas
    for titulo, num, contenido in titulos:
        info = {
            'titulo': f'{titulo.capitalize()} {num.strip()}',
            'enlace': contenido.strip()
        }
        lista_titulos.append(info)
    print(titulos)
    return lista_titulos

In [11]:
tit = extraer_titulos(texto)

[('TITULO', 'I', 'GENERALIDADES'), ('TITULO', 'II', 'DE LA SOLUCION DE CONFLICTOS, DEL PROCEDIMIENTO PARA LAS SANCIONES, DE LOS RECURSOS Y DE LAS SANCIONES'), ('TITULO', 'III', 'UNIDADES INMOBILIARIAS CERRADAS'), ('TITULO', 'IV', 'DISPOSICIONES FINALES')]


In [12]:
tit

[{'titulo': 'Titulo I', 'enlace': 'GENERALIDADES'},
 {'titulo': 'Titulo II',
  'enlace': 'DE LA SOLUCION DE CONFLICTOS, DEL PROCEDIMIENTO PARA LAS SANCIONES, DE LOS RECURSOS Y DE LAS SANCIONES'},
 {'titulo': 'Titulo III', 'enlace': 'UNIDADES INMOBILIARIAS CERRADAS'},
 {'titulo': 'Titulo IV', 'enlace': 'DISPOSICIONES FINALES'}]

# Exportar

In [96]:
titulos = extraer_titulos(texto)

Objeto y Definiciones
Artículo 1. Objeto. La presente ley regula la forma especial de dominio, denominado propiedad horizontal, en la que concurren de
De la constitución del Régimen de Propiedad Horizontal
ARTÍCULO 4º. Constitución. Un edificio o conjunto se somete al régimen de propiedad horizontal 
De la extinción de la propiedad horizontal
Artículo 9º. Causales de extinción de la propiedad horizontal. La propiedad horizontal se extinguirá por al
De la reconstrucción del edificio o conjunto
ARTÍCULO 13. Reconstrucción obligatoria. Se procederá a la reconstrucción del edificio o conjunto en los 
De los bienes privados o de dominio particular
ARTÍCULO 16. Identificación de los bienes privados o de dominio particular. Los bienes privados o de do
De los bienes comunes
ARTÍCULO 19.  Reglamentado por el Decreto Nacional 1060 de 2009. Alcance y naturaleza. Los bienes, los elementos y zonas de un e
De los coeficientes de copropiedad
ARTÍCULO 25. Obligatoriedad y efectos. Modificado por el ar

In [97]:
titulos

[{'titulo': 'Titulo I',
  'capitulos': [{'capitulo': 'Capitulo I',
    'articulos': [{'articulo': 'articulo 1',
      'contenido': 'Objeto. La presente ley regula la forma especial de dominio, denominado propiedad horizontal, en la que concurren derechos de propiedad exclusiva sobre bienes privados y derechos de copropiedad sobre el terreno y los demás bienes comunes, con el fin de garantizar la seguridad y la convivencia pacífica en los inmuebles sometidos a ella, así como la función social de la propiedad.NOTA: Texto subrayado declarado EXEQUIBLE por la Corte Constitucional mediante Sentencia C-318 de 2002, "bajo el entendido de que los moradores no propietarios de inmuebles sometidos al régimen de propiedad horizontal podrán ejercer ante las autoridades internas del mismo el derecho de petición, así como el de ser oídos en las decisiones que puedan afectarlos, conforme a lo expuesto en la parte motiva de esta sentencia.\'\'Igualmente, se declara la exequibilidad de las normas acusad

In [83]:
articulos = extraer_articulos_y_capitulos(texto)

In [39]:
def guardar_diccionario_en_csv(lista_diccionarios, nombre_archivo):
    # Obtener las claves del primer diccionario como encabezados
    keys = lista_diccionarios[0].keys()
    
    # Abrir el archivo en modo de escritura
    with open(nombre_archivo, 'w', newline='', encoding='utf-8') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()  # Escribir la cabecera del CSV
        dict_writer.writerows(lista_diccionarios)  # Escribir los datos del diccionario

    print(f"Archivo guardado como {nombre_archivo}")

In [128]:
guardar_diccionario_en_csv(capitulos, 'capitulos.csv')

Archivo guardado como capitulos.csv


In [98]:
guardar_diccionario_en_csv(titulos,'titulos.csv')

Archivo guardado como titulos.csv
